<a href="https://colab.research.google.com/github/Shkev/Hotdog_notHotdog/blob/master/hotdog_nothotdog_nn_with_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import math
import tensorflow as tf
import os
import numpy as np
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from IPython import display

print(tf.__version__)

#hyperparam
training_iterations = 10

HOTDOG_PATH = "/content/drive/My Drive/Colab Notebooks/ML/Hotdog_notHotdog/HotDogData/HotDog"
NOTHOTDOG_PATH = "/content/drive/My Drive/Colab Notebooks/ML/Hotdog_notHotdog/HotDogData/notHotDog"

2.1.0-rc1


#Defining Functions

In [0]:
#not used in model
def y_labels(indices, depth):
    return tf.one_hot(indices, depth)     #creates one_hot encoding for data; tensor size = (# of elements in index) x (depth)

In [0]:
def import_data(image_path):
  image_contents = tf.io.read_file(filename=image_path)   #reads image file
  modified_image = tf.image.decode_jpeg(contents=image_contents, channels=1)   #decodes jpeg image to tensor (uint8)
  cropped_and_modified_image_tensor = tf.image.resize_with_crop_or_pad(modified_image, 100, 100)   #resizes image
  image_tensor = tf.cast(cropped_and_modified_image_tensor, dtype=tf.float32)   #casts reshaped modified image tensor as a float32
  return image_tensor

#Creating Model


Reading in data.
Splitting Data into Training/Validation and Test set.


In [0]:
img_array = []
labels = []
num_hotdog = sum([len(files) for r, d, files in os.walk(HOTDOG_PATH)])
num_nothotdog = sum([len(files) for r, d, files in os.walk(NOTHOTDOG_PATH)])

In [0]:
for item in os.listdir(HOTDOG_PATH):
  img_path = os.path.join(HOTDOG_PATH, item)
  img_tensor = import_data(img_path)
  img_tensor = np.asarray(img_tensor) / 255. #scaling down pixel values to be between 0 and 1
  img_array.append(img_tensor)
  labels.append(1)

for item in os.listdir(NOTHOTDOG_PATH):
  img_path = os.path.join(NOTHOTDOG_PATH, item)
  img_tensor = import_data(img_path)
  img_tensor = np.asarray(img_tensor) / 255.
  img_array.append(img_tensor)
  labels.append(0)

In [0]:
img_array = np.array(img_array)
labels = np.array(labels)
img_array, labels = shuffle(img_array, labels) #shuffles two arrays together to keep items aligned with each other

Splitting Data into training, testing and validation sets

In [0]:
training_split_index = math.ceil(0.7*(num_hotdog + num_nothotdog))
training_images = img_array[:training_split_index]
training_labels = labels[:training_split_index]
testing_set = img_array[training_split_index:]
test_labels = labels[training_split_index:]

"""training_val_split_index = math.ceil(0.6*training_split_index)
training_set = training_images[:training_val_split_index]
training_labels = training_labels_set[:training_val_split_index]
validation_set = training_images[training_val_split_index:]
validation_labels = training_labels_set[training_val_split_index:]"""

'training_val_split_index = math.ceil(0.6*training_split_index)\ntraining_set = training_images[:training_val_split_index]\ntraining_labels = training_labels_set[:training_val_split_index]\nvalidation_set = training_images[training_val_split_index:]\nvalidation_labels = training_labels_set[training_val_split_index:]'

Creating Model

In [0]:
"""Using This model"""
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(16, 3, activation='relu', input_shape=(100, 100, 1)))
model.add(tf.keras.layers.Conv2D(32, 3, activation='relu'))
model.add(tf.keras.layers.Conv2D(64, 3, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu', kernel_constraint=tf.keras.constraints.max_norm(3)))
model.add(tf.keras.layers.Dense(32, activation='relu', kernel_constraint=tf.keras.constraints.max_norm(3)))
model.add(tf.keras.layers.Dense(16, activation='relu', kernel_constraint=tf.keras.constraints.max_norm(3)))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [0]:
"""model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(100, 100, 1)))
model.add(tf.keras.layers.Conv2D(32, 3, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=3))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(256, activation='relu', kernel_constraint=tf.keras.constraints.max_norm(2)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(128, activation='relu', kernel_constraint=tf.keras.constraints.max_norm(3)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(64, activation='relu', kernel_constraint=tf.keras.constraints.max_norm(3)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())

#output layer
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))"""

In [0]:
"""model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(16, 3, activation='relu', input_shape=(100, 100, 1)))
model.add(tf.keras.layers.MaxPooling2D(2))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu', kernel_constraint=tf.keras.constraints.max_norm(3)))
model.add(tf.keras.layers.Dense(16, activation='relu', kernel_constraint=tf.keras.constraints.max_norm(3)))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))"""

In [0]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

Fitting Model

In [0]:
model.summary()

In [0]:
history = model.fit(training_images,
                    training_labels,
                    epochs=10,
                    batch_size=50,
                    validation_split=0.4,
                    verbose=1)

Train on 109 samples, validate on 74 samples
Epoch 1/10
109/109 [==============================] - 1s 5ms/sample - loss: 0.8790 - accuracy: 0.3945 - val_loss: 0.6971 - val_accuracy: 0.2973
Epoch 2/10
109/109 [==============================] - 0s 318us/sample - loss: 0.6902 - accuracy: 0.5321 - val_loss: 0.6820 - val_accuracy: 0.6892
Epoch 3/10
109/109 [==============================] - 0s 273us/sample - loss: 0.6831 - accuracy: 0.6055 - val_loss: 0.6758 - val_accuracy: 0.6892
Epoch 4/10
109/109 [==============================] - 0s 282us/sample - loss: 0.6774 - accuracy: 0.6055 - val_loss: 0.6685 - val_accuracy: 0.6892
Epoch 5/10
109/109 [==============================] - 0s 283us/sample - loss: 0.6722 - accuracy: 0.6055 - val_loss: 0.6616 - val_accuracy: 0.6892
Epoch 6/10
109/109 [==============================] - 0s 277us/sample - loss: 0.6663 - accuracy: 0.6055 - val_loss: 0.6578 - val_accuracy: 0.6892
Epoch 7/10
109/109 [==============================] - 0s 277us/sample - loss: 0.6

In [0]:
history.history['accuracy']

Saving Model

In [0]:
model.save("/content/drive/My Drive/Colab Notebooks/ML/Hotdog_notHotdog/Saved_Model")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /content/drive/My Drive/Colab Notebooks/ML/Hotdog_notHotdog/Saved_Model/assets


In [0]:
model.load_weights("/content/drive/My Drive/Colab Notebooks/ML/Hotdog_notHotdog/Saved_Model/assets")

##Testing Model


In [0]:
test_loss, test_acc = model.evaluate(testing_set, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

78/78 - 0s - loss: 0.7353 - accuracy: 0.5128

Test accuracy: 0.51282054


###Making Predictions

In [0]:
predictions = model.predict(testing_set)

In [0]:
display.display(predictions, test_labels)

array([[0.60500574],
       [0.60637987],
       [0.60453945],
       [0.60508   ],
       [0.60442805],
       [0.60509676],
       [0.60641116],
       [0.60412556],
       [0.60357714],
       [0.6062592 ],
       [0.60265285],
       [0.6032412 ],
       [0.60624087],
       [0.6060382 ],
       [0.6064284 ],
       [0.60474277],
       [0.60452247],
       [0.6067172 ],
       [0.60292244],
       [0.6052927 ],
       [0.60321647],
       [0.6042203 ],
       [0.6060348 ],
       [0.6026124 ],
       [0.6060215 ],
       [0.603773  ],
       [0.60479337],
       [0.6064613 ],
       [0.60269195],
       [0.6047648 ],
       [0.6062867 ],
       [0.60395193],
       [0.6034429 ],
       [0.6066105 ],
       [0.60462445],
       [0.60458267],
       [0.603094  ],
       [0.6034218 ],
       [0.6049829 ],
       [0.604404  ],
       [0.60614693],
       [0.60590076],
       [0.6053628 ],
       [0.60276705],
       [0.60493046],
       [0.6063842 ],
       [0.60395336],
       [0.605

array([0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1])

#Making Predictions based on Other Images

Importing the image(s)

In [0]:
new_image_arr = []
new_labels = []

In [0]:
def import_extra_image(IMG_PATH, label, new_image_arr, new_labels):
  img_tensor = import_data(IMG_PATH)
  img_tensor = np.asarray(img_tensor) / 255. #scaling down pixel values to be between 0 and 1
  new_image_arr.append(img_tensor)
  new_labels.append(label)

In [0]:
import_extra_image("/content/drive/My Drive/Random stuff/Hydrangeas.jpg", 0, new_image_arr, new_labels)
new_image_arr = np.array(new_image_arr)
new_labels = np.array(new_labels)

In [0]:
display.display(new_image_arr)

In [0]:
predictions = model.predict(new_image_arr)
display.display(predictions)

array([[0.9745206]], dtype=float32)

In [0]:
for item in predictions:
  if item >=0.50:
    print("Hotdog!")
  else:
    print("Not Hotdog :(")

Hotdog!
